In [ ]:
import numpy as np
import pandas as pd

# Dataset: FoodAPS National Household Food Acquisition and Purchase Survey

This dataset contains the results of the USDA's National Household Food Acquisition and Purchase Survey (FoodAPS). It
collects information about "foods purchased or otherwise acquired for consumption at home and away from home, including
foods acquired through food and nutrition assistance programs."


In [ ]:
hh_df = pd.read_csv("data/faps_household_puf.csv")

fah_df = pd.read_csv("data/faps_fahnutrients.csv")
fafh_df = pd.read_csv("data/faps_fafhnutrient_puf.csv")

In [ ]:
hh_df = hh_df.sort_values(by="hhnum")

fah_df = fah_df.sort_values(by="hhnum")
fafh_df = fafh_df.sort_values(by="hhnum")

In [ ]:
hh_df = hh_df[(hh_df["numguests"] == 0) & (hh_df["hhsizechange"] == 0)]

In [ ]:
import enum


class TargetGroup(enum.Enum):
    NON_SNAP_HOUSEHOLD_LESS_FPG = 1
    NON_SNAP_HOUSEHOLD_MORE_FPG = 2
    NON_SNAP_HOUSEHOLD_GREATER_FPG = 3
    SNAP_HOUSEHOLD = 4


class SNAPNowAdmin(enum.Enum):
    NO_MATCH = 0
    CONFIRMED_SNAP = 1
    CONFIRMED_NON_SNAP = 2
    VALID_SKIP = -996


hh_df = hh_df[
    (hh_df["targetgroup"] == TargetGroup.SNAP_HOUSEHOLD.value)
    & (hh_df["snapnowadmin"] == SNAPNowAdmin.CONFIRMED_SNAP.value)
]

In [ ]:
hh_nums = hh_df["hhnum"]

In [ ]:
fah_df = fafh_df[fafh_df["hhnum"].isin(hh_nums)]
fafh_df = fafh_df[fafh_df["hhnum"].isin(hh_nums)]